In [6]:
import hsfs
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

connection = hsfs.connection()
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [2]:
books = fs.get_feature_group('books', version=1)

In [18]:
books_df = books.select_except(['book_title', 'publisher']).read().limit(50000).toPandas()

In [15]:
# Gotta normalize this dataframe, otherwise I all 1s
# try with standard deviation
# normalization and one-hot encoding of authors can happen at the transformation levels.
normalized_df=(books_df-books_df.mean())/(books_df.std())

In [16]:
cos_similarity = cosine_similarity(normalized_df)

In [17]:
pd.DataFrame(cos_similarity)[0].nlargest(n=6)

0        1.000000
39027    0.999999
3402     0.999989
21926    0.999980
4534     0.999980
8087     0.999932
Name: 0, dtype: float64

In [19]:
isbn = books_df['isbn']

In [28]:
isbn[21926]

'0935216677'